In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.session import SparkSession

In [2]:
conf = SparkConf().setAppName("0430_example").setMaster("local")
sc = SparkContext(conf=conf)

In [3]:
spark = SparkSession(sc)

In [4]:
sc

<SparkContext master=local appName=0430_example>

# RDD 이용하기

In [5]:
data = sc.textFile("./example.csv")

In [8]:
parsed_data = data.map(lambda x: (int(x.split(',')[0]), float(x.split(',')[2])))

res = parsed_data.reduceByKey(lambda x, y: round(x + y)).sortBy(lambda x : x[1], ascending=False)
res.collect()

[(68, 6378),
 (73, 6207),
 (39, 6198),
 (54, 6066),
 (71, 5995),
 (2, 5993),
 (97, 5979),
 (46, 5964),
 (42, 5692),
 (59, 5646),
 (41, 5638),
 (0, 5525),
 (8, 5518),
 (85, 5507),
 (61, 5497),
 (32, 5491),
 (58, 5438),
 (63, 5412),
 (15, 5410),
 (6, 5404),
 (92, 5374),
 (70, 5371),
 (43, 5369),
 (72, 5339),
 (34, 5329),
 (9, 5324),
 (55, 5300),
 (90, 5291),
 (64, 5290),
 (93, 5266),
 (24, 5259),
 (33, 5257),
 (62, 5254),
 (26, 5250),
 (52, 5247),
 (87, 5212),
 (40, 5185),
 (35, 5155),
 (11, 5149),
 (65, 5143),
 (69, 5119),
 (81, 5118),
 (25, 5062),
 (19, 5059),
 (60, 5041),
 (29, 5040),
 (17, 5035),
 (22, 5017),
 (28, 5001),
 (30, 4991),
 (16, 4981),
 (51, 4974),
 (1, 4958),
 (53, 4947),
 (18, 4922),
 (27, 4918),
 (86, 4908),
 (76, 4902),
 (38, 4899),
 (95, 4878),
 (89, 4852),
 (20, 4836),
 (88, 4830),
 (4, 4818),
 (82, 4815),
 (10, 4815),
 (31, 4766),
 (44, 4757),
 (7, 4754),
 (37, 4737),
 (14, 4734),
 (80, 4724),
 (21, 4708),
 (56, 4703),
 (66, 4683),
 (12, 4666),
 (3, 4659),
 (84, 46

# DataFrame 이용하기

In [9]:
df = spark.read.csv("./example.csv")

In [10]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [12]:
df_withSchema = spark.read.option("inferSchema", "true").csv("./example.csv")

In [13]:
df_withSchema.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: double (nullable = true)



In [20]:
df_withSchema = df.withColumnRenamed("_c0", "customerId")\
                .withColumnRenamed("_c1", "productId")\
                .withColumnRenamed("_c2", "spent")

In [21]:
df_withSchema.show()

+----------+---------+-----+
|customerId|productId|spent|
+----------+---------+-----+
|        44|     8602|37.19|
|        35|     5368|65.89|
|         2|     3391|40.64|
|        47|     6694|14.98|
|        29|      680|13.08|
|        91|     8900|24.59|
|        70|     3959|68.68|
|        85|     1733|28.53|
|        53|     9900|83.55|
|        14|     1505| 4.32|
|        51|     3378|19.80|
|        42|     6926|57.77|
|         2|     4424|55.77|
|        79|     9291|33.17|
|        50|     3901|23.57|
|        20|     6633| 6.49|
|        15|     6148|65.53|
|        44|     8331|99.19|
|         5|     3505|64.18|
|        48|     5539|32.42|
+----------+---------+-----+
only showing top 20 rows



In [25]:
from pyspark.sql import functions as F

df_withSchema.groupBy("customerId")\
  .agg(F.round(F.sum("spent"), 2).alias("totalSpent"))\
  .orderBy("totalSpent", ascending=False)\
  .show()

+----------+----------+
|customerId|totalSpent|
+----------+----------+
|        68|   6375.45|
|        73|    6206.2|
|        39|   6193.11|
|        54|   6065.39|
|        71|   5995.66|
|         2|   5994.59|
|        97|   5977.19|
|        46|   5963.11|
|        42|   5696.84|
|        59|   5642.89|
|        41|   5637.62|
|         0|   5524.95|
|         8|   5517.24|
|        85|   5503.43|
|        61|   5497.48|
|        32|   5496.05|
|        58|   5437.73|
|        63|   5415.15|
|        15|   5413.51|
|         6|   5397.88|
+----------+----------+
only showing top 20 rows



In [28]:
df_withSchema.createOrReplaceTempView("CUSTOMER")

In [31]:
data = spark.sql("select * from CUSTOMER")

res = spark.sql("""
                select customerId, round(sum(spent), 2) AS totalSpent
                from CUSTOMER
                group by customerId
                order by totalSpent desc
                """)
res.show()

+----------+----------+
|customerId|totalSpent|
+----------+----------+
|        68|   6375.45|
|        73|    6206.2|
|        39|   6193.11|
|        54|   6065.39|
|        71|   5995.66|
|         2|   5994.59|
|        97|   5977.19|
|        46|   5963.11|
|        42|   5696.84|
|        59|   5642.89|
|        41|   5637.62|
|         0|   5524.95|
|         8|   5517.24|
|        85|   5503.43|
|        61|   5497.48|
|        32|   5496.05|
|        58|   5437.73|
|        63|   5415.15|
|        15|   5413.51|
|         6|   5397.88|
+----------+----------+
only showing top 20 rows

